# Project 5: Optimizing Evacuation Routes using Real-Time Traffic Information

Song May | US-DSI-10 | 02.21.2020

In [202]:
import pandas as pd
import spacy
import os
import numpy as np
import datetime
import warnings
import os
import geocoder
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt
%matplotlib inline

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', -1)

In [169]:
tweets = pd.read_csv('data/final_data_with_predictions.csv')

In [170]:
tweets.head()

,dates&time,user,tweet,class
0,2017-08-31 21:34:27+00:00,DallasPD,"Reminder: Texas State Law prohibiting texting and driving goes into effect September 1, 2017",0
1,2017-08-31 21:03:18+00:00,DallasPD,Brotherhood for the Fallen Charity Event,0
2,2017-08-31 17:16:43+00:00,DallasPD,"FREE #TRINITY EVENT - Artisan Fair Market - September 2, 2017 #LaborDayWeekend",0
3,2017-08-31 17:13:36+00:00,DallasPD,Our thoughts and prayers are with the Sacramento County Sheriff's Department and the family of Deputy Sheriff Robert French.,0
4,2017-08-31 17:10:47+00:00,DallasPD,How can Dallas residents help animals affected by #HurricaneHarvey? http://fb.me/90nmndc7A,0


In [171]:
useful_twts = tweets[tweets['class'] == 1]

In [172]:
useful_twts.shape

(103, 4)

In [173]:
useful_twts.head(103)

,dates&time,user,tweet,class
239,2017-09-01 04:36:40+00:00,EPPOLICE,"I-10 East at Airway, scheduled road work, all lanes closed, follow detour, clearing time 5 am.",1
240,2017-09-01 04:09:53+00:00,EPPOLICE,"I-10 East at Mesa, schedule road work, all lanes closed, follow detour, clearing time 6 am.",1
242,2017-09-01 00:39:43+00:00,EPPOLICE,"I-10 East @US-54, Collision, Right Lane Closed, Backup To Porfirio Diaz, Clearing Time 1 Hour.",1
247,2017-08-31 23:10:23+00:00,EPPOLICE,"I-10 East @Schuster, Collision, Right Shoulder Closed, Backup To Executive, Clearing Time 1 Hour.",1
248,2017-08-31 22:51:34+00:00,EPPOLICE,"I-10 West @Trowbridge, Collision, Right Lane Closed, Backup To Hawkins, Clearing Time 1 Hour.",1
250,2017-08-31 22:28:02+00:00,EPPOLICE,"I-10 East @Kansas, Collision, Center lane closed, Backup to Asarco, Clearing time 1 hr, Alt Rte",1
253,2017-08-31 16:18:28+00:00,EPPOLICE,"I-10 West @Zaragoza, Stalled vehicle, Left lane closed, Backup to Don Haskins, Clearing time 1 hr",1
256,2017-08-31 15:21:25+00:00,EPPOLICE,"I-10 East at Asarco, stalled vehicle, center lane closed, backup to Executive Center, clearing time 1 hr",1
258,2017-08-31 14:39:20+00:00,EPPOLICE,"I-10 West at Lomaland, collision, left two lanes closed, backup to Lee Trevino, clearing time 1 hr.",1
260,2017-08-31 14:16:54+00:00,EPPOLICE,"I-10 East at Sunland Park, collision, right lane closed, minor backup, clearing time 1 hr.",1


In [174]:
useful_twts.reset_index()

,index,dates&time,user,tweet,class
0,239,2017-09-01 04:36:40+00:00,EPPOLICE,"I-10 East at Airway, scheduled road work, all lanes closed, follow detour, clearing time 5 am.",1
1,240,2017-09-01 04:09:53+00:00,EPPOLICE,"I-10 East at Mesa, schedule road work, all lanes closed, follow detour, clearing time 6 am.",1
2,242,2017-09-01 00:39:43+00:00,EPPOLICE,"I-10 East @US-54, Collision, Right Lane Closed, Backup To Porfirio Diaz, Clearing Time 1 Hour.",1
3,247,2017-08-31 23:10:23+00:00,EPPOLICE,"I-10 East @Schuster, Collision, Right Shoulder Closed, Backup To Executive, Clearing Time 1 Hour.",1
4,248,2017-08-31 22:51:34+00:00,EPPOLICE,"I-10 West @Trowbridge, Collision, Right Lane Closed, Backup To Hawkins, Clearing Time 1 Hour.",1
5,250,2017-08-31 22:28:02+00:00,EPPOLICE,"I-10 East @Kansas, Collision, Center lane closed, Backup to Asarco, Clearing time 1 hr, Alt Rte",1
6,253,2017-08-31 16:18:28+00:00,EPPOLICE,"I-10 West @Zaragoza, Stalled vehicle, Left lane closed, Backup to Don Haskins, Clearing time 1 hr",1
7,256,2017-08-31 15:21:25+00:00,EPPOLICE,"I-10 East at Asarco, stalled vehicle, center lane closed, backup to Executive Center, clearing time 1 hr",1
8,258,2017-08-31 14:39:20+00:00,EPPOLICE,"I-10 West at Lomaland, collision, left two lanes closed, backup to Lee Trevino, clearing time 1 hr.",1
9,260,2017-08-31 14:16:54+00:00,EPPOLICE,"I-10 East at Sunland Park, collision, right lane closed, minor backup, clearing time 1 hr.",1


In [178]:
useful_twts.drop(['dates&time','user'],axis = 1,inplace = True)

In [180]:
useful_twts.head()

,tweet,class
239,"I-10 East at Airway, scheduled road work, all lanes closed, follow detour, clearing time 5 am.",1
240,"I-10 East at Mesa, schedule road work, all lanes closed, follow detour, clearing time 6 am.",1
242,"I-10 East @US-54, Collision, Right Lane Closed, Backup To Porfirio Diaz, Clearing Time 1 Hour.",1
247,"I-10 East @Schuster, Collision, Right Shoulder Closed, Backup To Executive, Clearing Time 1 Hour.",1
248,"I-10 West @Trowbridge, Collision, Right Lane Closed, Backup To Hawkins, Clearing Time 1 Hour.",1


In [212]:
location = ['I-10 East at Airway, Texas',
 'I-10 East at Mesa, Texas',
 'I-10 East at Asarco, Texas',
 'I-10 West at Lomaland, Texas',
 'I-10 East at Sunland, Texas', 
 'I-10 West at Hawkins, Texas']

In [228]:
location_sample = pd.DataFrame(location)

In [229]:
location_sample.rename(columns={0:'locations'}, inplace =True) 

In [230]:
location_sample.head()

,locations
0,"I-10 East at Airway, Texas"
1,"I-10 East at Mesa, Texas"
2,"I-10 East at Asarco, Texas"
3,"I-10 West at Lomaland, Texas"
4,"I-10 East at Sunland, Texas"


In [216]:
# Credit for this cell goes to team Jasmine Vasandani, Phillip Dibert, and Eric Kropf.
coordinates = []
for i in range(len(locations['locations'])):
    g = geocoder.arcgis(locations['locations'][i])
    coordinates.append(tuple(g.latlng))

In [231]:
location_sample['latlng'] = coordinates

In [232]:
location_sample.head()

,locations,latlng
0,"I-10 East at Airway, Texas","(31.77851238297871, -106.39274056382978)"
1,"I-10 East at Mesa, Texas","(31.76263800000001, -106.49052)"
2,"I-10 East at Asarco, Texas","(31.152728536211036, -97.45027245045877)"
3,"I-10 West at Lomaland, Texas","(31.744404000000007, -106.33187249999999)"
4,"I-10 East at Sunland, Texas","(31.816627713233906, -106.55121538620033)"


In [221]:
import os
from bokeh.io import show,output_file
from bokeh.models import (GMapPlot, GMapOptions, ColumnDataSource,
                         Circle, DataRange1d, PanTool,
                         WheelZoomTool, BoxSelectTool)

In [233]:
location_sample.head()

,locations,latlng
0,"I-10 East at Airway, Texas","(31.77851238297871, -106.39274056382978)"
1,"I-10 East at Mesa, Texas","(31.76263800000001, -106.49052)"
2,"I-10 East at Asarco, Texas","(31.152728536211036, -97.45027245045877)"
3,"I-10 West at Lomaland, Texas","(31.744404000000007, -106.33187249999999)"
4,"I-10 East at Sunland, Texas","(31.816627713233906, -106.55121538620033)"


In [223]:
map_options = GMapOptions(lat = 31.76263800000001, lng = -106.49052,
                         map_type = 'roadmap', zoom = 3)

In [225]:
#api_key = os.environ['API_KEY']

In [234]:
location_sample.to_csv('location_sample.csv')

In [235]:
locations = pd.read_csv('locations.csv')

In [239]:
locations.head()

,location
0,"I-10 East at Airway, Texas"
1,"I-10 East at Mesa, Texas"
2,"I-10 East at Asarco, Texas"
3,"I-10 West at Lomaland, Texas"
4,"I-10 East at Sunland, Texas"


In [240]:
coordinates = []
for i in range(len(locations['location'])):
    g = geocoder.arcgis(locations['location'][i])
    coordinates.append(tuple(g.latlng))

In [241]:
locations['latlng'] = coordinates

In [267]:
locations.shape

(43, 2)

In [243]:
locations.to_csv('locations_w_LL.csv')

In [264]:
la_lng = pd.read_csv('lat_lng.csv')

In [266]:
la_lng.head()

,lat,lng
0,"31.77851238297871,",-106.392741
1,"31.76263800000001,",-106.490520
2,"31.152728536211036,",-97.450272
3,"31.744404000000007,",-106.331873
4,"31.816627713233906,",-106.551215
